In [1]:
import math
from disk_relations import *
from btree import *
from queryprocessing import *
from create_sample_databases import *

create_sample_databases has a function to create a database with two relations

In [2]:
# Create a sample database
db1 = createDatabase1("univ")

### B+Trees
We can print out the tuples and also the indexes as follows.

In [3]:
db1.getRelation("instructor").printTuples()

Relation instructor contains 6 blocks
Block No. 0, Type: RelationBlock: ('10101', 'Srinivasan', 'Comp. Sci.', '65000'), ('12121', 'Wu', 'Finance', '90000')
Block No. 1, Type: RelationBlock: ('15151', 'Mozart', 'Music', '40000'), ('22222', 'Einstein', 'Physics', '95000')
Block No. 2, Type: RelationBlock: ('32343', 'El Said', 'History', '60000'), ('33456', 'Gold', 'Physics', '87000')
Block No. 3, Type: RelationBlock: ('45565', 'Katz', 'Comp. Sci.', '75000'), ('58583', 'Califieri', 'History', '62000')
Block No. 4, Type: RelationBlock: ('76543', 'Singh', 'Finance', '80000'), ('76766', 'Crick', 'Biology', '72000')
Block No. 5, Type: RelationBlock: ('83821', 'Brandt', 'Comp. Sci.', '92000'), ('98345', 'Kim', 'Elec. Eng.', '80000')


In [4]:
db1.getIndex("instructor", "name").printTree()

Printing BTree Index on Relation instructor on Attribute name
--- Level 0 (root): 
	Block No. 16, Type: BTree, Parent: None: {Block 11}, Mozart, {Block 15}
--- Level 1: 
	Block No. 11, Type: BTree, Parent: 16: {Block 9}, Einstein, {Block 14}, Gold, {Block 12}
	Block No. 15, Type: BTree, Parent: 16: {Block 13}, Srinivasan, {Block 10}
--- Level 2: 
	Block No. 9, Type: BTree, Parent: 11: {Block 5, Tuple 0}, Brandt, {Block 3, Tuple 1}, Califieri, {Block 4, Tuple 1}, Crick, {Block 14}
	Block No. 14, Type: BTree, Parent: 11: {Block 1, Tuple 1}, Einstein, {Block 2, Tuple 0}, El Said, {Block 12}
	Block No. 12, Type: BTree, Parent: 11: {Block 2, Tuple 1}, Gold, {Block 3, Tuple 0}, Katz, {Block 5, Tuple 1}, Kim, {Block 13}
	Block No. 13, Type: BTree, Parent: 15: {Block 1, Tuple 0}, Mozart, {Block 4, Tuple 0}, Singh, {Block 10}
	Block No. 10, Type: BTree, Parent: 15: {Block 0, Tuple 0}, Srinivasan, {Block 0, Tuple 1}, Wu, None


Currently the B+-Tree is printed out level-by-level as shown here. {Block 36} is a pointer to the Block 36, whereas {Block 26, Tuple 0} is a pointer to a specific tuple.

In [10]:
DisplayBTree(db1.getIndex("instructor", "name"))

index = 0, blockNo = 9
index = 1, blockNo = 14
index = 2, blockNo = 12
index = 3, blockNo = 13


<btree.DisplayBTree instance at 0x7f34bc0f1cf8>

### Searching
The following snippet of code does a search using the tree, and prints out the resulting tuples. It also prints out the blocks that were retrieved during the search.

In [6]:
Globals.printBlockAccesses = True
results = db1.getIndex("instructor", "name").searchByRange("M", "S")
if results is not None and len(results) != 0:
    print "Results: " + " ".join([str(ptr.getTuple()) for ptr in results])
else:
    print "No results found"
Globals.printBlockAccesses = False

Retrieving Block No. 16, Type: BTree, Parent: None: {Block 11}, Mozart, {Block 15}
Retrieving Block No. 11, Type: BTree, Parent: 16: {Block 9}, Einstein, {Block 14}, Gold, {Block 12}
Retrieving Block No. 12, Type: BTree, Parent: 11: {Block 2, Tuple 1}, Gold, {Block 3, Tuple 0}, Katz, {Block 5, Tuple 1}, Kim, {Block 13}
Retrieving Block No. 13, Type: BTree, Parent: 15: {Block 1, Tuple 0}, Mozart, {Block 4, Tuple 0}, Singh, {Block 10}
Retrieving Block No. 1, Type: RelationBlock: ('15151', 'Mozart', 'Music', '40000'), ('22222', 'Einstein', 'Physics', '95000')
Results: ('15151', 'Mozart', 'Music', '40000')


### Deleting 
The following code finds a tuple and then deletes it from the relation (and effectively the two B+-Trees on the relation. We print out the final trees.

In [7]:
deleteKey = "Srinivasan"
print "Deleting the entry for key " + deleteKey
index = db1.getIndex("instructor", "name")
results = index.searchByKey(deleteKey)
db1.getRelation("instructor").deleteTuple(results[0])
# The BTrees should have been adjusted automatically
#index.printTree()
#db1.getIndex("instructor", "dept_name").printTree()

Deleting the entry for key Srinivasan
Block No. 15, Type: BTree, Parent: 16: {Block 13}, Srinivasan, {Block 10}
** 0 - 13 - 10
** 2 - 10 - 10
Block No. 16, Type: BTree, Parent: None: {Block 11}, Mozart, {Block 15}
** 0 - 11 - 15
** 2 - 15 - 15


In [9]:
DisplayBTree(db1.getIndex("instructor", "dept_name"))

KeyError: ('instructor', 'dept_name')

**Task 1**: However, if I try to delete "Einstein", there is an error because the functionality to redistribute is missing (and you have to implement this functionality).

In [8]:
deleteKey = "Einstein"
index = db1.getIndex("instructor", "name")
results = index.searchByKey(deleteKey)
db1.getRelation("instructor").deleteTuple(results[0])
# The BTrees should have been adjusted automatically
index.printTree()
db1.getIndex("instructor", "dept_name").printTree()

Block No. 11, Type: BTree, Parent: None: {Block 9}, Einstein, {Block 14}, Gold, {Block 12}, Mozart, {Block 13}
** 0 - 9 - 14
** 2 - 14 - 14
Redistributing entries between Block No. 9, Type: BTree, Parent: 11: {Block 5, Tuple 0}, Brandt, {Block 3, Tuple 1}, Califieri, {Block 4, Tuple 1}, Crick, {Block 14} and Block No. 14, Type: BTree, Parent: 11: {Block 2, Tuple 0}, El Said, {Block 12}


ValueError: Functionality to be implemented

## Query Processing Operators
`queryprocessing.py` contains simple implementations of several different operators, including SequentialScan, Nested Loops Join, and Hash Join. The operators are implemented using the iterator interface, where every operator exposes a basic API: `init()`, `get_next()`, and `close()`. This allows us to chain together the operators in a variety of ways without having to worry about which operator is providing input to which operator.

*Note: The provided code has limited functionality. For example, the joins require SequentialScan as the inputs, but this is a limitation of the current code.*

The following shows the simplest SequentialScan operator.

In [ ]:
scan1 = SequentialScan(db1.getRelation("instructor"))
scan1.init()
for t in scan1.get_next():
    print "---> " + str(t)

The following shows an example of a NestedLoopsJoin, where we first set up two SequentialScan's and then use them as inputs to the NestedLoopsJoin opeartor.

In [ ]:
scan1 = SequentialScan(db1.getRelation("instructor"))
scan2 = SequentialScan(db1.getRelation("department"))
nl_join = NestedLoopsJoin(scan1, scan2, "dept_name", "dept_name")
nl_join.init()
for t in nl_join.get_next():
    print "---> " + str(t)

If I reexecute the above after turning on printBlockAccesses, I can see all the blocks that were read.

In [ ]:
Globals.printBlockAccesses = True
nl_join.init()
for t in nl_join.get_next():
    print "---> " + str(t)
Globals.printBlockAccesses = False

Similarly, basic HashJoin also works.

In [ ]:
hash_join = HashJoin(SequentialScan(db1.getRelation("instructor")), SequentialScan(db1.getRelation("department")), "dept_name", "dept_name", HashJoin.INNER_JOIN)
hash_join.init()
for t in hash_join.get_next():
        print "---> " + str(t)

**Tasks 2, 3, 4**: However, SortMergeJoin, and the Outer Hash Joins don't work. You are to implement that functionality so that the following commands work.

In [ ]:
hash_join = HashJoin(SequentialScan(db1.getRelation("instructor")), 
                         SequentialScan(db1.getRelation("department")), "dept_name", "dept_name", HashJoin.LEFT_OUTER_JOIN)
hash_join.init()
for t in hash_join.get_next():
        print "---> " + str(t)

In [ ]:
hash_join = HashJoin(SequentialScan(db1.getRelation("instructor")), 
                         SequentialScan(db1.getRelation("department")), "dept_name", "dept_name", HashJoin.RIGHT_OUTER_JOIN)
hash_join.init()
for t in hash_join.get_next():
        print "---> " + str(t)

In [ ]:
sm_join = SortMergeJoin(SequentialScan(db1.getRelation("instructor")), 
                         SequentialScan(db1.getRelation("department")), "dept_name", "dept_name")
sm_join.init()
for t in sm_join.get_next():
        print "---> " + str(t)

#### Aggregates
Finally, the basic aggregate operation works as shown below (it produces a single answer).


**Task 5**: However, the following "group by" aggregate portion doesn't work. You are to implement that.

In [ ]:
aggr = GroupByAggregate(SequentialScan(db1.getRelation("instructor")), "salary", GroupByAggregate.SUM)
aggr.init()
for t in aggr.get_next():
        print "---> " + str(t)

aggr = GroupByAggregate(SequentialScan(db1.getRelation("instructor")), "salary", GroupByAggregate.SUM, "dept_name")
aggr.init()
for t in aggr.get_next():
        print "---> " + str(t)

In [ ]:
aggr = GroupByAggregate(SequentialScan(db1.getRelation("instructor")), "salary", GroupByAggregate.MAX)
aggr.init()
for t in aggr.get_next():
        print "---> " + str(t)

In [ ]:
aggr = GroupByAggregate(SequentialScan(db1.getRelation("instructor")), "salary", GroupByAggregate.MIN)
aggr.init()
for t in aggr.get_next():
        print "---> " + str(t)

In [ ]:
aggr = GroupByAggregate(SequentialScan(db1.getRelation("dept_name")), "salary", GroupByAggregate.MAX)
aggr.init()
for t in aggr.get_next():
        print "---> " + str(t)

In [ ]:
aggr = GroupByAggregate(SequentialScan(db1.getRelation("instructor")), "dept_name", GroupByAggregate.MAX)
aggr.init()
for t in aggr.get_next():
        print "---> " + str(t)

In [ ]:
index_name

In [ ]:
index.printTree()

In [ ]:
results = index.searchByKey("El Said")

In [ ]:
results

In [ ]:
results[0].getBlock()

In [ ]:
db1.getRelation("instructor").deleteTuple(results[0])

In [ ]:
class Shout(object):
    def __init__(self, text):
        self.text = text

    def _repr_html_(self):
        return '<h3>Detailed Breakdown</h3><svg class="chart" width="1220" height="320"><g transform="translate(0, 20)"><text x="0" y="10" dy=".35em">2016-10-03</text></g> <g transform="translate(120, 20)"><rect width="1000" height="20" fill="#ddd"/></g><g transform="translate(266.997088628, 20)"><rect width="0.0348439930556" height="20" fill="#000"/></g><line x1="864" y1="16" x2="864" y2="46"  stroke="black" stroke-width="1"/></svg>'


In [ ]:
Shout

In [ ]:
Shout("HELLO")